In [1]:
import folium
import pandas as pd
import numpy as np
import os
import math
import json
import vincent

## Import data

In [2]:
sites = pd.read_csv('results/full_sites_info.txt', sep='\t',dtype=str)

In [3]:
sites['SITE_ID'] = sites['SITE_ID'].astype(int)
sites['LAT'] = sites['LAT'].astype(float)
sites['LNG'] = sites['LNG'].astype(float)
sites['SQ_FT'] = sites['SQ_FT'].astype(float)
sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
Unnamed: 0      100 non-null object
SITE_ID         100 non-null int64
INDUSTRY        100 non-null object
SUB_INDUSTRY    100 non-null object
SQ_FT           100 non-null float64
LAT             100 non-null float64
LNG             100 non-null float64
ADDRESS         100 non-null object
STATE           100 non-null object
ZIP_CODE        100 non-null object
dtypes: float64(3), int64(1), object(6)
memory usage: 7.9+ KB


In [4]:
sites.INDUSTRY.unique()

array(['Commercial Property', 'Education', 'Food Sales & Storage',
       'Light Industrial'], dtype=object)

In [5]:
sites_1 = sites.loc[sites['INDUSTRY'] == 'Commercial Property']
sites_2 = sites.loc[sites['INDUSTRY'] == 'Education']
sites_3 = sites.loc[sites['INDUSTRY'] == 'Food Sales & Storage']
sites_4 = sites.loc[sites['INDUSTRY'] == 'Light Industrial']

In [6]:
data = pd.read_csv('results/final_result.csv')
data['datetime'] = pd.to_datetime(data['datetime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6100 entries, 0 to 6099
Data columns (total 9 columns):
Unnamed: 0          6100 non-null int64
datetime            6100 non-null datetime64[ns]
predict_TS          6100 non-null float64
predict_residual    6100 non-null float64
predict_value       6100 non-null float64
SITE_ID             6100 non-null int64
value               6100 non-null float64
TS                  6100 non-null float64
residual            6100 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 429.0 KB


# Create json plots

In [7]:
jsons = {}
grouped = data.groupby('SITE_ID')
for g in grouped.groups:
    df = grouped.get_group(g)[['datetime','value','predict_value']]
    df = df.loc[df['datetime'] <= '2012-11-10']
    df['datetime'] = df['datetime'].dt.strftime("%b %d")
    df['value'] = df['value'] /1000
    df['predict_value'] = df['predict_value'] /1000
    df = df.set_index('datetime')
    lines_plot = vincent.Line(df,width=500,height=300)
    lines_plot.scales[0].type = 'ordinal'
    lines_plot.axis_titles(x='Time', y='Power usage (*1000 kWh)')
    lines_plot.legend(title='Real vs Predicted')
    lines_json = lines_plot.to_json()
    jsons[g] = lines_json

# Create map with json plots

In [9]:
# Make an empty map
m = folium.Map(location = [sites['LAT'].mean(), sites['LNG'].mean()], zoom_start = 3.5)

layer_1 = folium.FeatureGroup(name = 'Commercial Property')
for lat,lon,name in zip(sites_1['LAT'],sites_1['LNG'],sites_1['SITE_ID']):
    layer_1.add_child(folium.Marker(location=[lat,lon],
                                    popup=folium.Popup(max_width = 700).add_child(folium.Vega(jsons[name], height=350, width=640)), 
                                    icon=folium.Icon(icon='shopping-bag', prefix = 'fa', color = 'red')))

layer_2 = folium.FeatureGroup(name = 'Education')
for lat,lon,name in zip(sites_2['LAT'],sites_2['LNG'],sites_2['SITE_ID']):
    layer_2.add_child(folium.Marker(location=[lat,lon],
                                    popup=folium.Popup(max_width = 700).add_child(folium.Vega(jsons[name], height=350, width=640)),
                                    icon=folium.Icon(icon='graduation-cap', prefix = 'fa', color = 'blue')))

layer_3 = folium.FeatureGroup(name = 'Food Sales & Storage')
for lat,lon,name in zip(sites_3['LAT'],sites_3['LNG'],sites_3['SITE_ID']):
    layer_3.add_child(folium.Marker(location=[lat,lon],
                                    popup=folium.Popup(max_width = 700).add_child(folium.Vega(jsons[name], height=350, width=640)), 
                                    icon=folium.Icon(icon='shopping-cart', prefix = 'fa', color = 'orange')))

layer_4 = folium.FeatureGroup(name = 'Light Industrial')
for lat,lon,name in zip(sites_4['LAT'],sites_4['LNG'],sites_4['SITE_ID']):
    layer_4.add_child(folium.Marker(location=[lat,lon],
                                    popup=folium.Popup(max_width = 700).add_child(folium.Vega(jsons[name], height=350, width=640)), 
                                    icon=folium.Icon(icon='industry', prefix = 'fa', color = 'green')))

In [10]:
m.add_child(layer_1)
m.add_child(layer_2)
m.add_child(layer_3)
m.add_child(layer_4)
m.add_child(folium.LayerControl())

In [11]:
m.save(outfile='map.html')